<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Carga_Archivos_Barra_Progresos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carga de datos con barra de progreso

In [10]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar datos
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Detectar tipo de instrumento según nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets de carga y salida
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
barra_progreso = widgets.IntProgress(value=0, min=0, max=1, description='Progreso:', bar_style='info')

# Función principal de carga con barra de progreso
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        total = len(archivos)
        barra_progreso.max = total
        barra_progreso.value = 0
        barra_progreso.bar_style = 'info'
        display(barra_progreso)

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"⏳ Cargando: {nombre_archivo} ({i}/{total})")

                if not instrumento:
                    print(f"❌ Instrumento no reconocido: {nombre_archivo}")
                    barra_progreso.value = i
                    continue

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

            barra_progreso.value = i

        barra_progreso.bar_style = 'success'
        print("✅ Carga finalizada.")
        mostrar_menu()

# Menú de visualización de DataFrames
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

#Carga de datos con barra de progresos y avance porcentual

In [11]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar datos
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Detectar tipo de instrumento según nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets de carga y salida
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
barra_progreso = widgets.FloatProgress(value=0, min=0, max=100, description='Progreso:', bar_style='info')
etiqueta_progreso = widgets.Label(value="0% completado")

# Función principal de carga con barra de progreso con porcentaje
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        total = len(archivos)
        barra_progreso.value = 0
        barra_progreso.bar_style = 'info'
        etiqueta_progreso.value = "0% completado"
        display(barra_progreso, etiqueta_progreso)

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"⏳ Cargando: {nombre_archivo} ({i}/{total})")

                if not instrumento:
                    print(f"❌ Instrumento no reconocido: {nombre_archivo}")
                    porcentaje = (i / total) * 100
                    barra_progreso.value = porcentaje
                    etiqueta_progreso.value = f"{porcentaje:.0f}% completado"
                    continue

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = "✅ Carga completada al 100%"
        print("✅ Carga finalizada.")
        mostrar_menu()

# Menú de visualización de DataFrames
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)


FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: EXTENSOMETRO (CSV)


,FECHA,EXTENSOMETRO,COTA_EXCAV._(MSNM),PROFUNDIDAD,Z/COTA_RELEV.,DIFERENCIAS_(MM),ACUMULADO_(MM),COTA_FINAL_EXCAV._(MSNM)
0,03/04/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
1,12/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
2,30/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
3,11/06/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
4,04/07/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0


# Version Mejorada de Carga de Datos con Selectores y botones

In [17]:
import pandas as pd
import io
import base64
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Configuración inicial ---
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return "puntos_fijos_mi"  # Valor por defecto si no se especifica
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Crear widgets globales
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
output_accion = widgets.Output()

# Los selectores y botones se crean una sola vez, fuera del menú y nunca dentro de un contexto de output.
instrumento_selector = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='300px')
)
origen_selector = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='200px')
)
boton_ver = widgets.Button(
    description='👁️ Ver Datos',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
boton_descargar = widgets.Button(
    description='💾 Descargar',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        barra_progreso = widgets.FloatProgress(
            value=0, min=0, max=100, description='Progreso:', bar_style='info',
            layout=widgets.Layout(width='50%')
        )
        etiqueta_progreso = widgets.Label(value="0% completado")
        display(barra_progreso, etiqueta_progreso)

        total = len(archivos)
        archivos_exitosos = 0

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"📄 Procesando: {nombre_archivo}")

                if not instrumento:
                    print(f"⚠️ No se pudo detectar el instrumento para: {nombre_archivo}")
                    porcentaje = (i / total) * 100
                    barra_progreso.value = porcentaje
                    etiqueta_progreso.value = f"{porcentaje:.0f}% completado"
                    continue

                # Procesar archivo según extensión
                if extension == 'csv':
                    try:
                        try:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                        except UnicodeDecodeError:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='latin-1')

                        if not df.empty:
                            datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                            print(f"✅ CSV cargado para {instrumento}: {len(df)} filas")
                            archivos_exitosos += 1
                        else:
                            print(f"⚠️ Archivo CSV vacío: {nombre_archivo}")
                    except Exception as e:
                        print(f"❌ Error al leer CSV {nombre_archivo}: {str(e)}")

                elif extension == 'xlsx':
                    try:
                        df = pd.read_excel(io.BytesIO(contenido))
                        if not df.empty:
                            datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                            print(f"✅ XLSX cargado para {instrumento}: {len(df)} filas")
                            archivos_exitosos += 1
                        else:
                            print(f"⚠️ Archivo XLSX vacío: {nombre_archivo}")
                    except Exception as e:
                        print(f"❌ Error al leer XLSX {nombre_archivo}: {str(e)}")
                else:
                    print(f"⚠️ Extensión no soportada: {extension}")

            except Exception as e:
                print(f"❌ Error general al procesar {nombre_archivo}: {str(e)}")

            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = f"✅ Carga completada: {archivos_exitosos}/{total} archivos procesados exitosamente"

        print(f"\n📊 Resumen de datos cargados:")
        for inst in instrumentos:
            csv_count = len(datos_csv[inst])
            xlsx_count = len(datos_xlsx[inst])
            if csv_count > 0 or xlsx_count > 0:
                print(f"  • {inst}: {csv_count} filas CSV, {xlsx_count} filas XLSX")

def ver_datos(b):
    with output_accion:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == "csv" else datos_xlsx[instrumento]
        print(f"📋 Mostrando datos de: {instrumento} ({origen})")
        print("=" * 50)
        if df.empty:
            print("⚠️ No hay datos disponibles para este instrumento y origen.")
            print("💡 Asegúrate de que el archivo se haya cargado correctamente.")
        else:
            print(f"📊 Total de filas: {len(df)}")
            print(f"📊 Total de columnas: {len(df.columns)}")
            print(f"📊 Columnas: {list(df.columns)}")
            print("\n🔍 Primeras 20 filas:")
            print("-" * 50)
            display(df.head(20))

def descargar_datos(b):
    with output_accion:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == "csv" else datos_xlsx[instrumento]
        print(f"💾 Preparando descarga: {instrumento} ({origen})")
        print("=" * 50)
        if df.empty:
            print("⚠️ No hay datos disponibles para descargar.")
            print("💡 Primero carga archivos que contengan datos para este instrumento.")
            return
        try:
            fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
            extension = 'csv' if origen == 'csv' else 'xlsx'
            nombre_archivo = f"{instrumento}_{origen}_{fecha_actual}.{extension}"
            buffer = io.BytesIO()
            if extension == 'csv':
                df.to_csv(buffer, index=False, encoding='utf-8')
                mime = "text/csv"
            else:
                df.to_excel(buffer, index=False, engine='openpyxl')
                mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            html = f'''
            <div style="padding: 10px; border: 2px solid #4CAF50; border-radius: 5px; background-color: #f9f9f9;">
                <h3>📥 Descarga Lista</h3>
                <p><strong>Archivo:</strong> {nombre_archivo}</p>
                <p><strong>Filas:</strong> {len(df)} | <strong>Columnas:</strong> {len(df.columns)}</p>
                <a download="{nombre_archivo}"
                   href="data:{mime};base64,{b64}"
                   target="_blank"
                   style="display: inline-block; padding: 10px 20px; background-color: #4CAF50; color: white; text-decoration: none; border-radius: 5px; font-weight: bold;">
                   📥 Hacer clic para descargar
                </a>
            </div>
            '''
            display(HTML(html))
            print("✅ Enlace de descarga generado exitosamente.")
            print("💡 Haz clic en el enlace y elige dónde guardar el archivo.")
        except Exception as e:
            print(f"❌ Error al generar la descarga: {str(e)}")

# Conectar eventos a los botones solo una vez
boton_ver.on_click(ver_datos)
boton_descargar.on_click(descargar_datos)
upload_widget.observe(cargar_archivos, names='value')

# Mostrar la interfaz
print("🚀 Sistema de Carga de Datos de Instrumentos")
print("=" * 50)
print("📁 Formatos soportados: CSV, XLSX")
print("🔧 Instrumentos disponibles:")
for i, inst in enumerate(instrumentos, 1):
    print(f"  {i}. {inst}")
print("\n📤 Selecciona uno o más archivos para comenzar:")

display(upload_widget)
display(output)
display(widgets.HBox([instrumento_selector, origen_selector]))
display(widgets.HBox([boton_ver, boton_descargar]))
display(output_accion)

🚀 Sistema de Carga de Datos de Instrumentos
📁 Formatos soportados: CSV, XLSX
🔧 Instrumentos disponibles:
  1. puntos_fijos_mi
  2. puntos_fijos_md
  3. inclinometros
  4. asentamiento
  5. piezometros_electricos
  6. piezometros_casagrande
  7. freatimetros
  8. extensometro

📤 Selecciona uno o más archivos para comenzar:


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

Output()

#Version Mejorada con Selectores Botones y boton Guardar

In [22]:
import pandas as pd
import io
import base64
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Configuración y estilos ---
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]
datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return "puntos_fijos_mi"
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets globales para mantener la UI estable
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'},
    layout=widgets.Layout(width="350px")
)
output_carga = widgets.Output()
output_tabla = widgets.Output()

instrumento_selector = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='260px')
)
origen_selector = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='160px')
)
boton_ver = widgets.Button(
    description='👁️ Ver',
    button_style='success',
    icon='eye',
    layout=widgets.Layout(width='100px')
)
boton_descargar = widgets.Button(
    description='💾 Descargar',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='130px')
)

# --- Función de carga y progreso ---
def cargar_archivos(change):
    with output_carga:
        clear_output(wait=True)
        archivos = upload_widget.value
        if not archivos:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No se subió ningún archivo.</div>"))
            return

        barra_progreso = widgets.FloatProgress(
            value=0, min=0, max=100, description='Progreso:', bar_style='info',
            layout=widgets.Layout(width='80%')
        )
        etiqueta_progreso = widgets.Label(value="0% completado")
        display(barra_progreso, etiqueta_progreso)

        total = len(archivos)
        archivos_exitosos = 0

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)
                contenido = archivo_info['content']

                if not instrumento:
                    display(HTML(f"<span style='color:#b71c1c;'>❌ Instrumento no reconocido en archivo: {nombre_archivo}</span>"))
                    continue

                if extension == 'csv':
                    try:
                        try:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                        except UnicodeDecodeError:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='latin-1')
                        if not df.empty:
                            datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como CSV ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ CSV vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo CSV {nombre_archivo}: {str(e)}</span>"))
                elif extension == 'xlsx':
                    try:
                        df = pd.read_excel(io.BytesIO(contenido))
                        if not df.empty:
                            datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como XLSX ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ XLSX vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo XLSX {nombre_archivo}: {str(e)}</span>"))
                else:
                    display(HTML(f"<span style='color:#b71c1c;'>⚠️ Extensión no soportada: {nombre_archivo}</span>"))
            except Exception as e:
                display(HTML(f"<span style='color:#b71c1c;'>❌ Error general en {nombre_archivo}: {str(e)}</span>"))
            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = f"✅ {archivos_exitosos}/{total} archivos procesados exitosamente"
        resumen = "<ul>"
        for inst in instrumentos:
            csv_count = len(datos_csv[inst])
            xlsx_count = len(datos_xlsx[inst])
            if csv_count > 0 or xlsx_count > 0:
                resumen += f"<li><b>{inst}</b>: {csv_count} filas CSV, {xlsx_count} filas XLSX</li>"
        resumen += "</ul>"
        display(HTML(f"<div style='margin-top:10px;'><b>Resumen de datos:</b>{resumen}</div>"))

# --- Función para mostrar tabla o mensaje ---
def ver_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos disponibles para el instrumento y origen seleccionados.</div>"))
        else:
            display(HTML(f"<div style='margin-bottom:10px;'><b>{instrumento.replace('_',' ').title()} ({origen.upper()})</b> - <span style='color:#388e3c'>Filas: {len(df)} | Columnas: {len(df.columns)}</span></div>"))
            display(df.head(5))  # AJUSTE: solo 5 registros

# --- Función para descargar con enlace bonito ---
def descargar_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos para descargar.</div>"))
            return
        fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
        extension = 'csv' if origen == 'csv' else 'xlsx'
        nombre_archivo = f"{instrumento}_{origen}_{fecha_actual}.{extension}"
        buffer = io.BytesIO()
        try:
            if extension == 'csv':
                df.to_csv(buffer, index=False, encoding='utf-8')
                mime = "text/csv"
            else:
                # openpyxl es necesario para xlsx, pero en Colab/Jupyter viene instalado
                df.to_excel(buffer, index=False, engine='openpyxl')
                mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            # NOTA: El enlace debe estar en una sola línea sin saltos para funcionar bien
            html = f"""
            <div style='padding:12px 18px;background:#e3f2fd;border:1.5px solid #2196f3;border-radius:7px;max-width:420px;'>
                <b>Descarga lista:</b><br>
                <span style='color:#1565c0'><b>{nombre_archivo}</b></span><br>
                <a download="{nombre_archivo}" href="data:{mime};base64,{b64}" target="_blank" style="display:inline-block;margin-top:10px;padding:10px 18px;background:#2196f3;color:white;text-decoration:none;border-radius:4px;font-weight:bold;">
                   📥 Descargar archivo
                </a>
            </div>
            """
            display(HTML(html))
            display(HTML("<span style='color:#388e3c;'>✔️ Haz clic en el botón para guardar el archivo en tu PC.</span>"))
        except Exception as e:
            display(HTML(f"<span style='color:#b71c1c;'>❌ Error al preparar la descarga: {str(e)}</span>"))

# --- Conectar eventos (solo una vez) ---
upload_widget.observe(cargar_archivos, names='value')
boton_ver.on_click(ver_datos)
boton_descargar.on_click(descargar_datos)

# --- Mostrar interfaz visual limpia y separada ---
display(HTML("""
<div style='margin-bottom:15px;'>
    <h2 style='color:#1976d2;margin:0 0 4px 0;'>📈 Sistema de gestión de datos de instrumentos</h2>
    <span style='color:#555;'>Carga, visualización y descarga de archivos CSV/XLSX</span>
</div>
"""))
display(HTML("<b>1. Subí tus archivos CSV/XLSX:</b>"))
display(upload_widget)
display(output_carga)
display(HTML("<hr style='margin:20px 0 10px 0;'>"))
display(HTML("<b>2. Seleccioná instrumento y origen de datos:</b>"))
display(widgets.HBox([instrumento_selector, origen_selector, boton_ver, boton_descargar]))
display(output_tabla)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', layout=Layout(width='350px'), multiple…

Output()

Output()